### 获取气候数据

因为直接全部下载会导致数据量过大，所以每下载一个就进行处理，只保留烟台区域的数据

#### 需求第三方库：

`numpy`, `pandas`, `tqdm`

In [11]:
# 初始化
import os
import numpy as np
import pandas as pd
import ftplib as ftp
from tqdm import tqdm

# 连接ftp站点
ftpSite = ftp.FTP()
ftpSite.connect(host="ftp2.tpdc.ac.cn", port=6201)
ftpSite.login(user="download_38710462", passwd="59317538")

print(ftpSite.getwelcome())
# print(ftpSite.dir())
# print(ftpSite.nlst('Data_forcing_03hr_010deg/SRad')[2:])

220---------- Welcome to Pure-FTPd [privsep] [TLS] ----------
220-You are user number 119 of 10000 allowed.
220-Local time is now 14:51. Server port: 6201.
220-IPv6 connections are also welcome on this server.
220 You will be disconnected after 15 minutes of inactivity.


In [9]:
# 下载测试

ftpSite.retrbinary('retr Data_forcing_03hr_010deg/SRad/srad_CMFD_V0106_B-01_03hr_010deg_197910.nc.gz',open('t1.nc.gz','wb').write,8192)
# ftpSite.retrbinary('retr Data_forcing_03hr_010deg/SRad/srad_CMFD_V0106_B-01_03hr_010deg_197910.nc.gz',open('t2.nc.gz','wb').write,8192)

'226-File successfully transferred\n226 2.428 seconds (measured here), 5.81 Mbytes per second'

In [ ]:
# 保存参数设置

dataPath  = '../data/climData/'         # 数据保存路径
cachePath = '../data/climData/cache/'   # 压缩包和nc文件暂存路径
timeRange = [199201,202312]                 # 时间段
latRange  = [36.45, 38.45]              # latitude Range
lonRange  = [119.55, 122.05]            # longitude Range

In [8]:
pathName   = 'Data_forcing_03hr_010deg/'
folderList = ['Temp', 'Prec', 'Pres', 'SRad', 'SHum', 'LRad', 'Wind']

withIn     = lambda val, range: True if val >= min(range) and val <= max(range) else False

fileCount  = 0
for folderName in folderList:
    path      = pathName + folderName + '/'
    fileCount += len(ftpSite.nlst(path)[2:])
pbar = tqdm(total=fileCount)

print(fileCount)
for folderName in folderList:
    path     = pathName + folderName + '/'
    fileList = ftpSite.nlst(path)[2:]
    for file in fileList:
        ftpSite.retrbinary('retr {}'.format(path + file), \
            open(cachePath + 'cache.nc.gz', 'wb').write)
        fileName = file.split('.nc.gz')[0]
        time = int(fileName[-6:])
        if not withIn(time, timeRange):
            continue
        
        
        

3360


In [13]:
from time import sleep
pbarT = tqdm(total=100)
for i in range(100):
    pbarT.update(i)
    sleep(0.5)
pbarT.close()

ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2